# Results in tables

In [21]:
import pandas as pd
import os
from collections import defaultdict
from IPython.display import display

In [28]:
def highlight_extremes(s: pd.Series):
    max_val = s.max()
    min_val = s.min()
    styles = []
    for v in s:
        if v == max_val:
            styles.append('background-color: lightgreen; font-weight: bold')
        elif v == min_val:
            styles.append('background-color: lightcoral; font-weight: bold')
        else:
            styles.append('')
    return styles

In [29]:
def display_experiment_results(results):
    if isinstance(results, pd.DataFrame):
        styled = results.style.apply(highlight_extremes, axis=0)
        display(styled)
        return
    
    if all(isinstance(v, pd.DataFrame) for v in results.values()):
        for name, df in results.items():
            print(f"\n=== {name} ===")
            styled = df.style.apply(highlight_extremes, axis=0)
            display(styled)
        return
    
    for model_type, exps in results.items():
        print(f"\n##### MODEL: {model_type} #####")
        for exp_name, df in exps.items():
            print(f"\n--- {exp_name} ---")
            styled = df.style.apply(highlight_extremes, axis=0)
            display(styled)

In [30]:
results_folder = "results"
model_types = ["cnn14_logstft", "logmel_cnn14"]
method_names = ["saliency", "gradcam", "lime", "shap"]
exp_types_mapping = {
    "cnn14_logstft": ["clean"],
    "logmel_cnn14": ["clean", "white", "room", "horse"]
}
folder_for_tables = "csvs"

In [31]:
csv_files = []

for model in model_types:
    for method in method_names:
        for exp in exp_types_mapping[model]:
            dir_path = os.path.join(
                results_folder,
                model,
                f"{method}_{exp}",
                folder_for_tables
            )
            if os.path.isdir(dir_path):
                for fname in os.listdir(dir_path):
                    if fname.endswith(".csv"):
                        csv_files.append(os.path.join(dir_path, fname))

In [32]:
experiment_results = {model: {} for model in model_types}

for fpath in csv_files:
    model_type = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(fpath))))
    experiment_name = os.path.basename(os.path.dirname(os.path.dirname(fpath)))
    mask_name = os.path.splitext(os.path.basename(fpath))[0]
    
    df = pd.read_csv(fpath)
    metrics = df.columns.difference(['sample', 'is_correct'])
    means = df[metrics].mean()
    
    if experiment_name not in experiment_results[model_type]:
        experiment_results[model_type][experiment_name] = {}
    experiment_results[model_type][experiment_name][mask_name] = means

for model_type, exps in experiment_results.items():
    for exp_name, mask_dict in exps.items():
        experiment_results[model_type][exp_name] = pd.DataFrame(mask_dict).T

## Mel model - clean

In [33]:
display_experiment_results(experiment_results["logmel_cnn14"]["saliency_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,34.762647,0.030776,19.500000,3.413026,0.923978,0.652500,0.176250
topK_30_pos,98.379270,0.000000,0.000000,9.171496,0.924232,0.015000,0.700006
minmax_pos,94.029495,0.049200,0.250000,9.025832,0.765674,0.087500,0.789556
topK_5_pos,97.663678,0.000000,0.000000,7.380256,0.765061,0.022500,0.949973
pos_thresh_50,97.721229,0.173707,0.250000,3.123119,0.033590,0.015000,0.998819
topK_50,98.340200,0.000000,0.000000,9.682342,0.925224,0.020000,0.500000
pos_thresh_25,97.134056,0.339988,0.500000,5.498956,0.267537,0.022500,0.990411
bin,98.346021,0.000000,0.000000,9.679453,0.925166,0.020000,0.501420
minmax,77.518508,3.182013,3.750000,10.372004,0.865298,0.272500,0.038222
topK_5,97.663678,0.000000,0.000000,7.380256,0.765061,0.022500,0.949973


In [34]:
display_experiment_results(experiment_results["logmel_cnn14"]["gradcam_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,38.935879,0.937880,11.250000,7.311478,0.620665,0.655000,0.376843
topK_30_pos,60.988887,0.592328,7.500000,7.798657,0.362573,0.447500,0.594134
minmax_pos,70.568771,0.675450,0.750000,9.305840,0.431521,0.362500,0.396169
topK_5_pos,88.365241,0.000586,2.500000,6.940669,0.079036,0.140000,0.889484
pos_thresh_50,60.330152,1.725173,4.750000,8.580231,0.365905,0.460000,0.539708
topK_50,50.483148,2.337507,4.250000,9.684185,0.525057,0.550000,0.499072
pos_thresh_25,37.203954,2.856096,12.250000,7.528248,0.599441,0.682500,0.347645
bin,24.415372,2.344471,21.500000,4.918489,0.719301,0.782500,0.195540
minmax,66.666616,0.772345,1.250000,10.150722,0.513954,0.392500,0.335306
topK_5,93.796811,0.000000,0.000000,7.403085,0.030442,0.095000,0.948799


In [36]:
display_experiment_results(experiment_results["logmel_cnn14"]["lime_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,30.268507,5.367577,13.750000,9.411879,0.827965,0.755000,0.460687
topK_30_pos,49.213681,4.776571,10.000000,9.252109,0.623068,0.550000,0.674052
minmax_pos,60.692678,1.596777,3.500000,9.738116,0.755739,0.437500,0.547097
topK_5_pos,72.956990,1.222816,2.000000,7.874178,0.158620,0.305000,0.913671
pos_thresh_50,58.130710,1.669558,3.000000,8.789097,0.493181,0.472500,0.748397
topK_50,30.978062,6.091994,13.750000,9.728370,0.826212,0.747500,0.475981
pos_thresh_25,34.795579,4.611255,9.500000,9.407741,0.801759,0.702500,0.562260
bin,8.220830,12.582630,29.250000,9.588828,0.900067,0.932500,0.198701
minmax,56.334667,2.253361,4.250000,10.071627,0.788149,0.487500,0.392208
topK_5,72.956990,1.222816,2.000000,7.874178,0.158620,0.305000,0.913671


In [37]:
display_experiment_results(experiment_results["logmel_cnn14"]["shap_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,98.489148,0.000000,0.000000,9.682342,0.927127,0.017500,0.500000
topK_30_pos,98.376756,0.000000,0.000000,9.171496,0.920322,0.015000,0.700006
minmax_pos,98.140790,0.000000,0.000000,9.325663,0.742942,0.017500,0.730384
topK_5_pos,97.529223,0.000000,0.000000,7.380256,0.644688,0.027500,0.949973
pos_thresh_50,97.753458,0.190183,0.250000,4.424030,0.059366,0.025000,0.996374
topK_50,98.489148,0.000000,0.000000,9.682342,0.927127,0.017500,0.500000
pos_thresh_25,97.999605,0.000000,0.000000,7.109134,0.514174,0.020000,0.955756
bin,98.448436,0.000000,0.000000,9.704492,0.927023,0.015000,0.488779
minmax,93.938898,0.000000,0.000000,10.366691,0.896360,0.080000,0.069193
topK_5,97.529223,0.000000,0.000000,7.380256,0.644688,0.027500,0.949973


## STFT model - clean

In [40]:
display_experiment_results(experiment_results["cnn14_logstft"]["saliency_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,48.375174,0.005567,23.000000,5.221556,0.804260,0.515000,0.246247
topK_30_pos,96.591307,0.693553,1.000000,10.091336,0.775904,0.027500,0.699991
minmax_pos,94.490105,0.376756,1.250000,9.997836,0.793775,0.045000,0.783133
topK_5_pos,96.526038,0.564025,0.750000,8.299783,0.616458,0.032500,0.949986
pos_thresh_50,96.525065,0.244557,0.500000,3.675487,0.031187,0.030000,0.999007
topK_50,96.391408,0.428297,0.500000,10.602145,0.791538,0.025000,0.499994
pos_thresh_25,95.673791,0.120974,1.000000,6.327209,0.220229,0.037500,0.991122
bin,96.423009,0.428138,0.500000,10.601499,0.791234,0.025000,0.500300
minmax,66.198419,6.311180,10.000000,11.292009,0.787943,0.345000,0.038136
topK_5,96.526038,0.564025,0.750000,8.299783,0.616458,0.032500,0.949986


In [41]:
display_experiment_results(experiment_results["cnn14_logstft"]["gradcam_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,31.095250,0.212617,22.750000,4.771013,0.724521,0.700000,0.224973
topK_30_pos,50.516011,0.218326,13.500000,6.357624,0.572838,0.520000,0.440970
minmax_pos,90.740430,0.087145,0.500000,9.345481,0.320162,0.100000,0.610099
topK_5_pos,84.389165,0.070849,4.750000,7.304025,0.191101,0.160000,0.835977
pos_thresh_50,89.263071,0.207852,0.250000,8.491924,0.267266,0.117500,0.762309
topK_50,59.905164,1.156712,4.000000,10.602260,0.585988,0.435000,0.499936
pos_thresh_25,73.915097,0.982882,2.250000,9.141945,0.398405,0.270000,0.600836
bin,54.052505,1.076550,9.250000,8.473753,0.474271,0.505000,0.451469
minmax,75.670870,0.522353,1.750000,11.139101,0.567426,0.262500,0.295463
topK_5,95.884257,0.070103,0.250000,8.300031,0.114918,0.042500,0.949974


In [42]:
display_experiment_results(experiment_results["cnn14_logstft"]["lime_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,55.774762,8.450201,15.500000,10.191675,0.765800,0.442500,0.473102
topK_30_pos,66.936226,5.281612,10.000000,10.118867,0.694406,0.345000,0.691540
minmax_pos,84.515637,0.381859,1.000000,10.526510,0.637870,0.182500,0.621597
topK_5_pos,89.532042,0.135247,0.750000,8.526989,0.243031,0.135000,0.936590
pos_thresh_50,84.201880,0.290252,0.750000,9.094784,0.433959,0.182500,0.869844
topK_50,58.199032,8.862266,14.750000,10.616330,0.765135,0.415000,0.492814
pos_thresh_25,69.090076,2.461060,5.500000,9.976185,0.677671,0.327500,0.699007
bin,32.426248,13.723769,25.000000,10.960349,0.788757,0.677500,0.272594
minmax,77.675293,1.586585,2.750000,11.106747,0.689437,0.237500,0.310267
topK_5,89.532042,0.135247,0.750000,8.526989,0.243031,0.135000,0.936590


In [43]:
display_experiment_results(experiment_results["cnn14_logstft"]["shap_clean"])

,AD,AG,AI,COMP,FF,FidIn,SPS
topK_50_pos,88.001383,0.378346,4.500000,9.515425,0.802138,0.117500,0.448744
topK_30_pos,97.378701,0.387124,0.500000,10.091336,0.763902,0.027500,0.699991
minmax_pos,96.696586,0.026467,0.250000,10.238195,0.455324,0.020000,0.733023
topK_5_pos,96.537593,0.030067,0.250000,8.299783,0.444992,0.017500,0.949986
pos_thresh_50,97.548443,0.169460,0.750000,4.935369,0.042822,0.010000,0.997583
topK_50,97.530267,0.377015,0.500000,10.602145,0.797199,0.020000,0.499994
pos_thresh_25,97.009564,0.000000,0.000000,7.850699,0.325784,0.020000,0.963614
bin,97.488084,0.376223,0.500000,10.618963,0.796336,0.020000,0.491457
minmax,92.499063,0.592648,1.250000,11.285742,0.758162,0.085000,0.072735
topK_5,96.537593,0.030067,0.250000,8.299783,0.444992,0.017500,0.949986
